<a href="https://colab.research.google.com/github/mgstockwell/water_well_prediction/blob/main/Groundwater_Load_CHIRPS_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.


### Authenticate and Initialize Earth Engine and BigQuery

Use service account to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. A separate set of BQ credentials created with additional setings and scopes.

In [1]:
# import the Earth Engine API
%load_ext google.cloud.bigquery
import os, sys, json, datetime
import pandas as pd
import ee
from google.colab import files, auth
from google.cloud import bigquery
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account as sa

google_project_id = 'msd8654-498-dev'
# Set the project id
os.environ.putenv('GOOGLE_CLOUD_PROJECT', google_project_id)

#check for key, upload if not there
if os.path.exists('/content/msd8654-498-dev-1a070409c971.json'):
  print('key exists')
else:
  f = files.upload()
KEY = '/content/msd8654-498-dev-1a070409c971.json'

# set credentials 
os.environ.putenv('GOOGLE_APPLICATION_CREDENTIALS', KEY)
service_account='my-first-app-92f7826c73ade84fa@msd8654-498-dev.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, KEY)


# Trigger the authentication flow if no key
#ee.Authenticate()

ee.Initialize(
  credentials=credentials,
  project=google_project_id,
  opt_url='https://earthengine-highvolume.googleapis.com'
)

# BigQuery section
# Set credentials and scopes 
# see https://developers.google.com/identity/protocols/oauth2/scopes#compute
bq_credentials = sa.Credentials.from_service_account_file(
      KEY, scopes=["https://www.googleapis.com/auth/cloud-platform"]
  )
bq_credentials.scopes.append('https://www.googleapis.com/auth/bigquery')
bq_credentials.scopes.append('https://www.googleapis.com/auth/cloud-platform')
bq_credentials.scopes.append('https://www.googleapis.com/auth/drive.metadata')
bq_credentials.scopes.append('https://www.googleapis.com/auth/compute')
print("key scopes:", bq_credentials.scopes)

bq_client = bigquery.Client(credentials=bq_credentials, project=google_project_id )

# set the bq credentials explicity 
# see https://googleapis.dev/python/bigquery/latest/magics.html
bigquery.magics.context.credentials = bq_credentials
bigquery.magics.context.project = google_project_id

datasets = list(bq_client.list_datasets())  # Make an API request.
project = bq_client.project

if datasets:
    print("Datasets in project {}:".format(project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("{} project does not contain any datasets.".format(project))

# create dataFrames for later to avoid error highlighting
df, df_state_cds = pd.DataFrame(), pd.DataFrame()

Saving msd8654-498-dev-1a070409c971.json to msd8654-498-dev-1a070409c971.json
key scopes: ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/drive.metadata', 'https://www.googleapis.com/auth/compute']
Datasets in project msd8654-498-dev:
	usgs


## Test the Earth Engine API

Test the API by printing the elevation of Mount Everest.

In [2]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


## BigQuery Upload Function

In [3]:
from google.cloud import storage
from google.cloud import bigquery
import urllib.request
import os, datetime

# note: google_project_name  & google_dataset_name at top (global)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the google storage bucket."""

    storage_client = storage.Client(project=google_project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to Storage Bucket {} successfully . {}".format(
            source_file_name, destination_blob_name, datetime.datetime.now()
        )
    )

# BigQuery to DataFrames via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [4]:
# Save output in a variable `df`
%%bigquery df
SELECT
          DISTINCT site_no,
          station_nm,
          site_tp_cd,
          lat_va,
          long_va,
          dec_lat_va,
          dec_long_va,
          district_cd,
          right(concat('0',state_cd),2) state_cd,
          county_cd,
          well_depth_va,
          well_depth_hue
        FROM
          `msd8654-498-dev.usgs.groundwater_sites_vis`
        WHERE
          well_depth_va IS NOT NULL
          and dec_lat_va is not null
          and dec_long_va is not null
          and cast(state_cd as int) <10
        ORDER BY state_cd, site_no

In [5]:
df

,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,district_cd,state_cd,county_cd,well_depth_va,well_depth_hue
0,301418087525401,Well FFF 8 301418087525401 Baldwin County Al,GW,301418,875254.2,30.238333,-87.881722,1,01,3,15.0,2
1,301418087530401,Well FFF 1 301418087530401 Baldwin County Al,GW,303333,875304,30.559360,-87.884437,1,01,3,32.0,2
2,301425087510501,Well FFF 3 301425087510501 Baldwin County Al,GW,301429,875121,30.241592,-87.855825,1,01,3,105.0,4
3,301430087443001,Well EEE 9 301430087443001 Baldwin County Al,GW,301456,874546,30.249092,-87.762766,1,01,3,87.0,3
4,301442088055901,Well UU 46 Mobile Cnty AL,GW,301442,880559,30.245204,-88.099721,1,01,97,43.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
117083,420209072432401,CT-SU 32,GW,420209,724324,42.035928,-72.722871,9,09,3,450.0,6
117084,420210073064101,CT-C 32,GW,420210,730641,42.036204,-73.110942,9,09,5,246.0,5
117085,420211073202101,CT-NOC 14,GW,420211,732021,42.036481,-73.338728,9,09,5,28.0,2
117086,420224073251101,CT-SY 25,GW,420224,732511,42.040092,-73.419286,9,09,5,400.0,6


In [6]:
# Save output in a variable `df_state_cds`

%%bigquery df_state_cds
SELECT
  state_fips_code,
  state_postal_abbreviation,
  state_name,
  state_gnisid
FROM
  `bigquery-public-data.census_utility.fips_codes_states`
  where cast(state_fips_code as INT)  <10


In [7]:
df_state_cds.head()

,state_fips_code,state_postal_abbreviation,state_name,state_gnisid
0,01,AL,Alabama,01779775
1,02,AK,Alaska,01785533
2,04,AZ,Arizona,01779777
3,05,AR,Arkansas,00068085
4,06,CA,California,01779778


### Get Precipitation from Earth Engine assets

[CHIRPS Pentad: Climate Hazards Group InfraRed Precipitation With Station Data (Version 2.0 Final)](https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD#description)
The image layer is made up of many polygons with a "band" value. The lat/long point will fall within a polygon, and the value of band (in this case annual precipitation in mm) will be returned.

  <p>Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS) is a 30+ year quasi-global rainfall dataset. CHIRPS incorporates 0.05° resolution satellite imagery with in-situ station data to create gridded rainfall time series for trend analysis and seasonal drought monitoring..</p>

```
Resolution
5566 meters

Bands

Name	Units	Min	Max	Description
precipitation	mm/pentad	0*	1072.43*	
Precipitation

* estimated min or max value
```

In [8]:
from time import sleep
import pandas as pd

# Check a known point
# Point (-75.7778, 40.4375) at 76m/px
# Pixels
# Lithology: Image (1 band)
# b1: 3
# ALOS mTPI: Image (1 band)
# AVE: -5

# load the Climate Hazard global rainfall dataset and summarize by year
# https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD#description
# https://spatialthoughts.com/2020/10/28/rainfall-data-gee/
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').select('precipitation')
year = 2020
startDate = ee.Date.fromYMD(year, 1, 1)
endDate = startDate.advance(1, 'year')
filtered = chirps.filter(ee.Filter.date(startDate, endDate))
precipitation = filtered.reduce(ee.Reducer.sum())
print(precipitation.getInfo())

def get_precip(long: float, lat: float):
  xy = ee.Geometry.Point([round(long,2),round(lat,2)])
  try:
    data = precipitation.sample(xy, 100, dropNulls=True).first().get('precipitation_sum').getInfo()
  except BaseException as err:
    data = precipitation.sample(xy, 1000, dropNulls=True).first().get('precipitation_sum').getInfo()
  return data

'''0 site_no        350160106324201
dec_lat_va           35.033358
dec_long_va        -106.545522
'''
print(datetime.datetime.now(), 'START')
print(get_precip(-74.54,40.03))

for j, row in df_state_cds.iterrows():
  state_postal_abbreviation = row["state_postal_abbreviation"]
  print("state_cd:", state_postal_abbreviation)
  df_filtered = df[df.state_cd==row["state_fips_code"]]
  filename = f'precip_{state_postal_abbreviation}.csv'
  file = open(filename,'w')
  for i, row in df_filtered.iterrows():
    try:
      val = get_precip(row["dec_long_va"], row["dec_lat_va"])
      file.writelines(str(i) + "," + row["site_no"] + "," + str(val) + '\n')
    except BaseException as err:
      print(f"   Unexpected {err}, {type(err)}")
      print('ERROR processed:',i,row.to_json(), val, " "*10, datetime.datetime.now())
      continue

    if ((i+1)%100==0): 
      print('processed:',i,row["site_no"], val,state_postal_abbreviation ," "*10, datetime.datetime.now())
      sleep(1)
  file.close()
  upload_blob('msd8654-498-dev-usgs',filename, filename)

print(datetime.datetime.now(), 'END')


{'type': 'Image', 'bands': [{'id': 'precipitation_sum', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
2022-06-04 22:20:02.319860 START
1138.1479024887085
state_cd: AL
processed: 99 301923087320701 1633.3540076613426 AL            2022-06-04 22:20:58.050101
processed: 199 302829087341101 1742.6012271046638 AL            2022-06-04 22:21:49.825165
processed: 299 303840087521501 1683.1498274803162 AL            2022-06-04 22:22:20.252483
processed: 399 305305087462901 1661.7092784643173 AL            2022-06-04 22:22:49.829541
processed: 499 310309087121701 1657.3142381310463 AL            2022-06-04 22:23:18.850785
processed: 599 310914085154601 1547.973560154438 AL            2022-06-04 22:23:47.821404
processed: 699 311406087274301 1564.7084249258041 AL            2022-06-04 22:24:16.101347
processed: 799 311832085451801 1720.1679192185402 AL            2022-06-04 22:24:44.953908
processed: 899 312133085520001 1628

RefreshError: ignored